In [ ]:
!pip install langchain-anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [langchain-anthropic]


In [12]:
import os
from langchain_anthropic import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain.memory import ChatMessageHistory

In [ ]:


# 대화형 Claude 모델 초기화
chat = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0.7)

In [28]:
# chat_history = ChatMessageHistory()

# # 📝 초기 대화 저장(선택)
# chat_history.add_user_message("안녕하세요. 제 이름은 산초입니다.")
# chat_history.add_ai_message("안녕하세요, 산초님! 무엇을 도와드릴까요?")


In [29]:
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "너는 친절한 한국어 비서야. 이전 대화 내용을 참고해서 질문에 답하며, 반드시 한국어로만 대답해."
    ),
    ("placeholder", "{chat_history}"),
    ("user", "{input}")
])


In [30]:
def limit_chat_history(history_obj, max_turns=3):
    """
    최근 'max_turns'번의 사용자-AI 대화만 남기기 (각 각 1개씩, 총 6개 메세지 유지)
    """
    max_messages = max_turns * 2  # 사용자 + AI = 2씩 곱함
    if len(history_obj.messages) > max_messages:
        history_obj.messages = history_obj.messages[-max_messages:]


In [31]:
from langchain_core.runnables import Runnable

# 대화 체인 구성
chain_with_history_limit = prompt | chat

def chat_with_bot(user_input):
    if user_input.strip().lower() == "리셋":
        chat_history.clear()
        return "✅ 대화 기록이 초기화되었습니다."

    elif user_input.strip().lower() == "히스토리":
        return show_chat_history()

    # 사용자 입력 저장
    chat_history.add_user_message(user_input)

    # 히스토리 제한 적용
    limit_chat_history(chat_history)

    try:
        # AI에 입력 보내고 응답 받기
        response = chain_with_history_limit.invoke({
            "input": user_input,
            "chat_history": chat_history.messages
        })
        
        # AI 응답 저장
        chat_history.add_ai_message(response.content)
        return response.content
    
    except Exception as e:
        return f"❌ 오류 발생: {str(e)}"


In [32]:
def show_chat_history():
    print("🕓 현재 대화 기록 (최근 3회 대화만 표시됨):")
    for i, message in enumerate(chat_history.messages):
        speaker = "👤 사용자" if isinstance(message, HumanMessage) else "🤖 AI"
        print(f"{i+1}. [{speaker}]: {message.content}")
    return ""

In [38]:
print(chat_with_bot("내이름은 산초야"))


안녕하세요, 산초님. 반갑습니다. 제가 도와드릴 수 있는 일이 있나요?


In [39]:
print(chat_with_bot("내이름은 뭐야?"))

네, 알겠습니다. 산초님의 이름이 맞습니다.


In [40]:
print(chat_with_bot("히스토리"))

🕓 현재 대화 기록 (최근 3회 대화만 표시됨):
1. [👤 사용자]: 내이름은 산초야
2. [🤖 AI]: 안녕하세요, 산초님. 반갑습니다. 제가 도와드릴 수 있는 일이 있나요?
3. [👤 사용자]: 내이름은 뭐야?
4. [🤖 AI]: 네, 알겠습니다. 산초님의 이름이 맞습니다.



In [37]:
print(chat_with_bot("리셋"))

✅ 대화 기록이 초기화되었습니다.


In [34]:
print(chat_with_bot("내 이름이 뭐야"))

죄송합니다. 제가 알고 있는 정보로는 당신의 이름이 무엇인지 정확히 알 수 없습니다. 앞서 말씀하신 것처럼 당신의 이름이 산초님이라고 들었지만, 다시 한번 묻고 싶습니다. 당신의 이름이 무엇인지 알려주시면 감사하겠습니다.
